In [1]:
# From: https://www.kaggle.com/c/dog-breed-identification/data

from __future__ import print_function, division

import os, sys
import argparse
import json
import numpy as np
import pprint
from PIL import Image

import pandas as pd
from pandas import Series, DataFrame

import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms, utils

from configs.config_train import get_cfg_defaults

In [2]:
def parse_args(**args):
    parser = argparse.ArgumentParser(description='dog breed')
    parser.add_argument("--cfg", type=str, default="configs/config_train.yaml", help="Configurations.")
    # return parser.parse_args(**args)          # for python
    return parser.parse_known_args(**args)    # for jupyter notebook

In [3]:
data_path = r'D:\GitWork\dog_breed\data'
print('Data path: ', data_path)

label_path = r'D:\GitWork\dog_breed\data\raw'
label_fname = 'labels.csv'

lbl_abspath = os.path.join(label_path, label_fname)
df = pd.read_csv(lbl_abspath)

columns = list(df.columns)
print('\nColumns: ', columns)
print()

print(df.info());print()
print(df.head())

Data path:  D:\GitWork\dog_breed\data

Columns:  ['id', 'breed']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10222 non-null  object
 1   breed   10222 non-null  object
dtypes: object(2)
memory usage: 159.8+ KB
None

                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [4]:
# Create train ids and valid ids

frac_for_train = 0.8

id_sers = Series.to_numpy(df["id"])
id_sers_num = id_sers.shape[0]
print('Num id series: ', id_sers_num)

train_num = int(float(id_sers_num) * float(frac_for_train))
valid_num = id_sers_num - train_num
print('\nNum for train: ', train_num)
print('Num for valid: ', valid_num)

train_ids = id_sers[:train_num]
valid_ids = id_sers[train_num:]
print('\nTrain ids:'); print('\n'.join(train_ids[:10]))

train_path = os.path.join(data_path, 'train')
train_files = [os.path.join(train_path, f+'.jpg') for f in train_ids]
valid_files = [os.path.join(train_path, f+'.jpg') for f in valid_ids]

print('\nTrain files:'); print('\n'.join(train_files[:10]))
print('\nValid files:'); print('\n'.join(valid_files[:10]))

ProcPath = r'D:\GitWork\dog_breed\data\processed'
np.save(os.path.join(ProcPath, "train_ids.npy"), train_files)
np.save(os.path.join(ProcPath, "valid_ids.npy"), valid_files)

Num id series:  10222

Num for train:  8177
Num for valid:  2045

Train ids:
000bec180eb18c7604dcecc8fe0dba07
001513dfcb2ffafc82cccf4d8bbaba97
001cdf01b096e06d78e9e5112d419397
00214f311d5d2247d5dfe4fe24b2303d
0021f9ceb3235effd7fcde7f7538ed62
002211c81b498ef88e1b40b9abf84e1d
00290d3e1fdd27226ba27a8ce248ce85
002a283a315af96eaea0e28e7163b21b
003df8b8a8b05244b1d920bb6cf451f9
0042188c895a2f14ef64a918ed9c7b64

Train files:
D:\GitWork\dog_breed\data\train\000bec180eb18c7604dcecc8fe0dba07.jpg
D:\GitWork\dog_breed\data\train\001513dfcb2ffafc82cccf4d8bbaba97.jpg
D:\GitWork\dog_breed\data\train\001cdf01b096e06d78e9e5112d419397.jpg
D:\GitWork\dog_breed\data\train\00214f311d5d2247d5dfe4fe24b2303d.jpg
D:\GitWork\dog_breed\data\train\0021f9ceb3235effd7fcde7f7538ed62.jpg
D:\GitWork\dog_breed\data\train\002211c81b498ef88e1b40b9abf84e1d.jpg
D:\GitWork\dog_breed\data\train\00290d3e1fdd27226ba27a8ce248ce85.jpg
D:\GitWork\dog_breed\data\train\002a283a315af96eaea0e28e7163b21b.jpg
D:\GitWork\dog_breed\data\t

In [5]:
# 建立品種名稱與編號的對應字典

breed_sers = Series.to_numpy(df['breed'])
breed_sers_num = breed_sers.shape[0]
print('Breed series num: ', breed_sers_num)
    
# 整理有多少種品種
breed_set = set(breed_sers)
breed_set_len = len(breed_set)
print('Breed set size: ', breed_set_len)

# 建構一個品種名稱與編號的對應字典
breed_list = list(breed_set)
breed_list.sort()
breed_dict = { v:i for i, v in enumerate(breed_list) }
breed_dict_len = len(breed_dict)

print('\nBreed dict len: ', breed_dict_len)
print(json.dumps(breed_dict, indent=4))

Breed series num:  10222
Breed set size:  120

Breed dict len:  120
{
    "affenpinscher": 0,
    "afghan_hound": 1,
    "african_hunting_dog": 2,
    "airedale": 3,
    "american_staffordshire_terrier": 4,
    "appenzeller": 5,
    "australian_terrier": 6,
    "basenji": 7,
    "basset": 8,
    "beagle": 9,
    "bedlington_terrier": 10,
    "bernese_mountain_dog": 11,
    "black-and-tan_coonhound": 12,
    "blenheim_spaniel": 13,
    "bloodhound": 14,
    "bluetick": 15,
    "border_collie": 16,
    "border_terrier": 17,
    "borzoi": 18,
    "boston_bull": 19,
    "bouvier_des_flandres": 20,
    "boxer": 21,
    "brabancon_griffon": 22,
    "briard": 23,
    "brittany_spaniel": 24,
    "bull_mastiff": 25,
    "cairn": 26,
    "cardigan": 27,
    "chesapeake_bay_retriever": 28,
    "chihuahua": 29,
    "chow": 30,
    "clumber": 31,
    "cocker_spaniel": 32,
    "collie": 33,
    "curly-coated_retriever": 34,
    "dandie_dinmont": 35,
    "dhole": 36,
    "dingo": 37,
    "doberman": 

In [6]:
# 將 label.csv 中所有的品種類別轉換成品種編號

train_bids = [breed_dict[df.loc[df['id']==i].breed.item()] for i in train_ids]
valid_bids = [breed_dict[df.loc[df['id']==i].breed.item()] for i in valid_ids]

def showBreedIds(ids, num=10):
    sample = ids[:num]
    breeds = [df.loc[df['id']==i].breed.item() for i in sample]
    bids = [breed_dict[i] for i in breeds]
    
    col_width = max(len(i) for i in breeds)
    print('id,breed,breed_id')
    for i in range(len(sample)):
        v = sample[i]
        breed = "".join(breeds[i].ljust(col_width))
        print('{}, {}, {}'.format(v, breed, bids[i]))
    return
        
showBreedIds(train_ids, num=10)
print(); print(train_bids[:10])

# print()
# showBreedIds(valid_ids, num=10)
# print(); print(valid_bids[:10])

id,breed,breed_id
000bec180eb18c7604dcecc8fe0dba07, boston_bull       , 19
001513dfcb2ffafc82cccf4d8bbaba97, dingo             , 37
001cdf01b096e06d78e9e5112d419397, pekinese          , 85
00214f311d5d2247d5dfe4fe24b2303d, bluetick          , 15
0021f9ceb3235effd7fcde7f7538ed62, golden_retriever  , 49
002211c81b498ef88e1b40b9abf84e1d, bedlington_terrier, 10
00290d3e1fdd27226ba27a8ce248ce85, bedlington_terrier, 10
002a283a315af96eaea0e28e7163b21b, borzoi            , 18
003df8b8a8b05244b1d920bb6cf451f9, basenji           , 7
0042188c895a2f14ef64a918ed9c7b64, scottish_deerhound, 97

[19, 37, 85, 15, 49, 10, 10, 18, 7, 97]


In [7]:
# Save labels as file
np.save(os.path.join(ProcPath, "train_labels.npy"), train_bids)
np.save(os.path.join(ProcPath, "valid_labels.npy"), valid_bids)

In [11]:
# Create Dataset

# Normalize
normalize = transforms.Normalize(
    mean = [0.485, 0.456, 0.406],
    std  = [0.229, 0.224, 0.225]
)

# Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])


def default_loader(imgPath, fname):
    img_pil = Image.open(os.path.join(imgPath, fname))
    img_pil = img_pil.resize((224,224))
    return img_pil


class myDataset(Dataset):
    
    def __init__(self, path, phase='train', transform=None):
        img_files = os.path.join(path, '{}_ids.npy'.format(phase))
        lbl_files = os.path.join(path, '{}_labels.npy'.format(phase))
        
        self.images = np.load(img_files)
        self.labels = np.load(lbl_files)
        self.transform = transform
        self.len = len(self.images)

    def __getitem__(self, index):
        img_file = self.images[index]
        
        img = Image.open(img_file)
        img = img.resize((224,224))

        if self.transform is not None:
            img = self.transform(img)
        
        lbl = self.labels[index]
        return img, lbl

    def __len__(self):
        return self.len
    
trainSet = myDataset(ProcPath, transform=transform)
trainLoader = DataLoader(trainSet, batch_size=100, shuffle=True)
print(trainLoader.shape)

AttributeError: 'DataLoader' object has no attribute 'shape'

In [ ]:
class MyDataset(Dataset):
    def __init__(self, txt, transform=None, target_transform=None, loader=default_loader):
        fh = open(txt, 'r')
        imgs = []
        for line in fh:
            line = line.strip('\n')
            line = line.rstrip()
            words = line.split()
            imgs.append((words[0],int(words[1])))
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = self.loader(fn)
        if self.transform is not None:
            img = self.transform(img)
        return img,label

    def __len__(self):
        return len(self.imgs)

train_data=MyDataset(txt='mnist_test.txt', transform=transforms.ToTensor())
data_loader = DataLoader(train_data, batch_size=100,shuffle=True)
print(len(data_loader))

In [10]:
# Normalize
normalize = transforms.Normalize(
    mean = [0.485, 0.456, 0.406],
    std  = [0.229, 0.224, 0.225]
)

# Transform
train_transform = transforms.Compose([
#     transforms.Scale(256),
#     transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])





In [11]:
train_data  = trainset()
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)

In [13]:
print(type(train_loader))

data_iter = iter(train_loader)
images, labels = data_iter.next()

print(type(images), type(labels))
print(images.size(), labels.size())

<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([4, 3, 224, 224]) torch.Size([4])


In [ ]:
def main():
    # arguments 
    args, _ = parse_args()
    print(args)

    cfg = get_cfg_defaults()
    cfg.merge_from_file(args.cfg)
    cfg.freeze()
    print(cfg)